# Simple Neural Collaborative Filtering (DMF-like)

Chúng ta sẽ xây dựng một mô hình Neural Collaborative Filtering đơn giản (tương tự Deep Matrix Factorization) sử dụng PyTorch.
Mô hình sẽ sử dụng cơ chế Learning Embeddings cho User và Item, sau đó kết hợp qua tích vô hướng (Dot Product) để dự đoán điểm đánh giá.

### Các bước thực hiện:
1.  Import thư viện và Load dữ liệu.
2.  Tiền xử lý: Mapping UserID và MovieID sang dạng chỉ số (index).
3.  Xây dựng PyTorch Dataset và DataLoader.
4.  Định nghĩa mô hình Neural Network.
5.  Huấn luyện (Training).
6.  Đánh giá (Evaluation).

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

# Kiểm tra GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Load dữ liệu
# Lưu ý: Điều chỉnh đường dẫn nếu cần thiết
train_path = 'ratings_train.csv'
test_path = 'ratings_test.csv'

# Đọc dữ liệu (chỉ lấy các cột cần thiết để tiết kiệm bộ nhớ nếu file lớn)
cols = ['userId', 'movieId', 'rating']
df_train = pd.read_csv(train_path, usecols=cols)
df_test = pd.read_csv(test_path, usecols=cols)

print(f"Train size: {len(df_train)}")
print(f"Test size: {len(df_test)}")
df_train.head()

In [ ]:
# Preprocessing: Encoding User and Movie IDs
# Chúng ta cần map userId và movieId về khoảng [0, N) để dùng trong Embedding Layer

# Gom tất cả user và item từ train và test để tạo mapping đầy đủ
all_users = pd.concat([df_train['userId'], df_test['userId']]).unique()
all_movies = pd.concat([df_train['movieId'], df_test['movieId']]).unique()

# Tạo encoder
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

user_encoder.fit(all_users)
movie_encoder.fit(all_movies)

# Transform dữ liệu
df_train['user_idx'] = user_encoder.transform(df_train['userId'])
df_train['movie_idx'] = movie_encoder.transform(df_train['movieId'])

df_test['user_idx'] = user_encoder.transform(df_test['userId'])
df_test['movie_idx'] = movie_encoder.transform(df_test['movieId'])

num_users = len(all_users)
num_movies = len(all_movies)

print(f"Number of Users: {num_users}")
print(f"Number of Movies: {num_movies}")

In [ ]:
class RatingDataset(Dataset):
    def __init__(self, user_indices, movie_indices, ratings):
        self.users = torch.tensor(user_indices, dtype=torch.long)
        self.movies = torch.tensor(movie_indices, dtype=torch.long)
        self.ratings = torch.tensor(ratings, dtype=torch.float32)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.users[idx], self.movies[idx], self.ratings[idx]

# Tạo Dataset và DataLoader
train_dataset = RatingDataset(df_train['user_idx'].values, df_train['movie_idx'].values, df_train['rating'].values)
test_dataset = RatingDataset(df_test['user_idx'].values, df_test['movie_idx'].values, df_test['rating'].values)

batch_size = 1024 # Batch size lớn giúp train nhanh hơn với dữ liệu lớn
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class SimpleDMF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=64, hidden_dims=[64, 32]):
        super(SimpleDMF, self).__init__()
        
        # User Embedding & MLP
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.user_layers = nn.ModuleList()
        input_dim = embedding_dim
        for dim in hidden_dims:
            self.user_layers.append(nn.Linear(input_dim, dim))
            self.user_layers.append(nn.ReLU())
            input_dim = dim
            
        # Item Embedding & MLP
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        self.item_layers = nn.ModuleList()
        input_dim = embedding_dim
        for dim in hidden_dims:
            self.item_layers.append(nn.Linear(input_dim, dim))
            self.item_layers.append(nn.ReLU())
            input_dim = dim
            
        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Embedding):
                nn.init.normal_(m.weight, std=0.01)

    def forward(self, user_indices, item_indices):
        # User Tower
        user_vec = self.user_embedding(user_indices)
        for layer in self.user_layers:
            user_vec = layer(user_vec)
            
        # Item Tower
        item_vec = self.item_embedding(item_indices)
        for layer in self.item_layers:
            item_vec = layer(item_vec)
            
        # Normalize vectors for Cosine Similarity (optional, but typical for DMF)
        # Hoặc dùng Dot Product trực tiếp. Ở đây dùng Dot Product đơn giản cho Regression.
        # Nếu muốn đúng chuẩn DMF paper thì dùng Cosine similarity. 
        # Tuy nhiên với rating 1-5, dot product tự học độ lớn sẽ dễ hội tụ hơn.
        
        interaction = (user_vec * item_vec).sum(dim=1)
        
        return interaction

# Khởi tạo mô hình
model = SimpleDMF(num_users, num_movies).to(device)
print(model)

# Loss và Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training Loop
num_epochs = 10
train_losses = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for users, movies, ratings in train_loader:
        users = users.to(device)
        movies = movies.to(device)
        ratings = ratings.to(device)
        
        optimizer.zero_grad()
        predictions = model(users, movies)
        loss = criterion(predictions, ratings)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * users.size(0)
        
    avg_loss = total_loss / len(train_dataset)
    train_losses.append(avg_loss)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

# Plot Loss
plt.plot(train_losses)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.show()

In [ ]:
# Evaluation
model.eval()
predictions_list = []
targets_list = []

with torch.no_grad():
    for users, movies, ratings in test_loader:
        users = users.to(device)
        movies = movies.to(device)
        
        preds = model(users, movies)
        
        predictions_list.extend(preds.cpu().numpy())
        targets_list.extend(ratings.numpy())

rmse = np.sqrt(mean_squared_error(targets_list, predictions_list))
print(f"Test RMSE: {rmse:.4f}")

# Example Predictions
df_result = pd.DataFrame({'Actual': targets_list[:10], 'Predicted': predictions_list[:10]})
print(df_result)